In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from numpy import around
import os
import math
files_path = "/content/drive/MyDrive/BioInfo/data/"
start_codons = ['ATG']
stop_codons = ['TAG', 'TAA', 'TGA']

Read file

In [ ]:
def getGenomes():
    _, _, files = next(os.walk(files_path))
    file_count = len(files)
    genomesArray = []
    num = 0
    for i in range(0,int(file_count/2)):
      num += 1;
      for record in SeqIO.parse(files_path + "bacterial" + str(num) + ".fasta", "fasta"):
          genomesArray.append(record)

    num = 0
    for i in range(int(file_count/2),int(file_count)):
      num += 1
      for record in SeqIO.parse(files_path + "mamalian" + str(num) + ".fasta", "fasta"):
          genomesArray.append(record)

    return genomesArray

Start and stop pairs

In [ ]:
def codoneStartToStop(frame, stride, x, length):
    found = False
    for j in range(len(frame[stride:])):
        x += frame[j + stride]
        length += 1
        if frame[j + stride:j + stride + 3] in stop_codons:
            x += frame[j + stride + 1] + frame[j + stride + 2]
            found = True
            length += 2
            break
    return found, x

In [ ]:
def codoneStopToStart(frame, stride, i, length):
    found = False
    x = ""
    for j in range(len(frame[stride:])):
        if frame[j + stride: j + stride + 3] in start_codons:
            x = frame[i: j + stride + 3]
            length += 3
            found = True
            j += 3
        elif frame[j + stride: j + stride + 3] in stop_codons:
            break
        length += 1
    return found, x

In [ ]:
def findCodons(arr, frame, globalArray, isStartToStop):
    for i in range(len(frame)):
        stride = i + 3
        length = 0
        if frame[i:stride] in globalArray:
            length += 3
            if isStartToStop:
                found, x = codoneStartToStop(frame, stride, frame[i:stride], length)
            else:
                found, x = codoneStopToStart(frame, stride, i, length)
            if found:
                arr.append(x)
        i += length

In [ ]:
def callThreeFrames(arr, sequence, isStartToStop):
    for i in range(3):
        frame = sequence[i::3]
        if isStartToStop:
            findCodons(arr, frame, start_codons, isStartToStop)
        else:
            findCodons(arr, frame, stop_codons, isStartToStop)

In [ ]:
def findCodonPairs(sequence, isStartToStop):
    genomesArr = []
    callThreeFrames(genomesArr, sequence, isStartToStop)
    rcSeq = sequence.reverse_complement()
    callThreeFrames(genomesArr, rcSeq, isStartToStop)
    return genomesArr

In [ ]:
def getPairs(genomes, isStartToStop):
    arr = []
    for i in range(len(genomes)):
        codonPairs = findCodonPairs(genomes[i].seq, isStartToStop)
        arr.append(codonPairs)
    return arr

Filtering fragments

In [ ]:
def filterByLength(array):
    filteredArr = []
    for value in array:
      if len(value) > 100:
        filteredArr.append(value)
    return filteredArr

Frequency calculation

In [ ]:
def getAllPossible(sequence, n1, n2):
    array = []
    i = 0
    while i < len(sequence):
        if sequence[i:i+n2] not in array:
            if len(sequence[i:i+n2]) == n2:
                array.append(sequence[i:i+n2])
        i += n1
    return array

In [ ]:
class Frequency:
    def __init__(self, code, freq) -> None:
        self.code = code
        self.freq = freq

In [ ]:
def getFrequencies(sequence, n1, n2):
    freqs = []
    allPossible = getAllPossible(sequence, n1, n2)
    for possible in allPossible:
        frequencies = Frequency(possible, sequence.count(possible) / len(sequence))
        freqs.append(frequencies)
    return freqs

In [ ]:
def concatSeqs(sequence):
    sum = Seq("")
    for item in sequence:
        sum += item
    return sum

In [ ]:
def getFrequenciessArr(genomes, codes1, codes2, n1, n2):
    arr = []
    for i in range(len(genomes)):
        concated = concatSeqs(codes1[i]) + concatSeqs(codes2[i])
        if concated:
            frequencies = getFrequencies(concated, n1, n2)
            arr.append(frequencies)
        else:
            arr.append([])
    return arr

Calculate distance

In [ ]:
def compareFrequencies(array1, array2):
    resultArr = []
    results = 0.0
    sameCodeItem = None
    for i in range(len(array1)):
        for x in array2:
          if x.code == array1[i].code:
            sameCodeItem = array1[i]
          if sameCodeItem:
            results += math.pow((array1[i].freq - sameCodeItem.freq), 2)
    if results != 0:
      return math.sqrt(results)
    else:
      return 0

Create Phylip matrix

In [ ]:
def createPhilypMatrix(genomes, freqs):
    philypMatrix = []
    for i in range(len(genomes)):
        philypMatrix.append([])
    for i in range(len(genomes)):
        for j in range(len(genomes)):
            comparedFrequencies = compareFrequencies(freqs[i], freqs[j])          
            philypMatrix[i].append(comparedFrequencies)
    return philypMatrix

In [ ]:
def printPhilypMatrix(genomes, matrix):
    print(len(genomes))
    for i in range(len(genomes)):
        print(genomes[i].id, *matrix[i])

In [ ]:
genomes = getGenomes()

Pateiktoje sekoje fasta formatu surastu visas start ir stop kodonų poras, tarp kurių nebutu stop kodono (ir tiesioginei sekai ir jos reverse komplementui). 

In [ ]:
codoneStartToStop = getPairs(genomes, True)

Kiekvienam stop kodonui parinkti toliausiai nuo jo esanti start kodoną (su salyga, kad tarp ju nera kito stop kodono)

In [ ]:
codoneStopToStart = getPairs(genomes, False)

Atfiltruokite visus fragmentus ("tai butu baltymų koduojancios sekos"), kurie trumpesni nei 100 fragmentų.

In [ ]:
for i in range(len(genomes)):
    codoneStartToStop[i] = filterByLength(codoneStartToStop[i])
    codoneStopToStart[i] = filterByLength(codoneStopToStart[i])

Parasykite funkcijas, kurios ivertintu kodonu ir dikodonu daznius (visi imanomi kodonai/dikodonai ir jų atitinkamas daznis  - gali buti nemazai nuliu, jei ju sekoje nerasite).

In [ ]:
codonFrequenciesArray = getFrequenciessArr(genomes, codoneStartToStop, codoneStopToStart, 1, 3)
dicodonFrequenciesArray = getFrequenciessArr(genomes, codoneStartToStop, codoneStopToStart, 3, 6)

Palyginkite kodonu bei dikodonu daznius tarp visu seku (atstumu matrica - kokia formule naudosite/kaip apskaiciuosite - parasykite ataskaitoje).

In [ ]:
codonsPhylipMatrix = createPhilypMatrix(genomes, codonFrequenciesArray)
dicodonsPhylipMatrix = createPhilypMatrix(genomes, dicodonFrequenciesArray)

In [ ]:
printPhilypMatrix(genomes,codonsPhylipMatrix)
printPhilypMatrix(genomes, dicodonsPhylipMatrix)

8
Lactococcus_phage 0.6076679644588743 0.6211032368046677 0.6438718862608998 0.5558457493238774 0.5750353732366184 0.545042205629218 0 0.5891022153398761
KM389305.1 0.4755986897885548 0.4529588456304246 0.4154866040842463 0.42722113733332345 0.4308426939013924 0.43887463704654206 0 0.3481194141674622
NC_028697.1 0.5358878004954647 0.4959502920961891 0.47042423492334484 0.5169692544043272 0.5935246650124189 0.5190677135901581 0 0.49194769834373875
KC821626.1 0.39621967718562084 0.35472261650855247 0.40919609756341735 0.3004080460204673 0.3373002137132277 0.37951094601938534 0 0.3785081971847041
coronavirus 0.4001662095164526 0.402828458462073 0.41189253948665466 0.3679126348787901 0.3977528056270681 0.370139508859677 0 0.39903998911006444
adenovirus 0.28767601288865274 0.3084858739013338 0.31093616442155897 0.2966037353429062 0.310818197480009 0.28469625481236693 0 0.2871692708190424
U18337.1 0 0 0 0 0 0 0 0
herpesvirus 0.3540644172599693 0.3410137126283061 0.37005691867847557 0.3968887